In [66]:
#import libraries needed
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from dotenv import load_dotenv
%matplotlib inline

In [67]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [68]:
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SECRET = os.getenv('PLAID_SECRET')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [69]:
INSTITUTION_ID = "ins_109508"

In [70]:
# Load .env into environmental variables
load_dotenv()
#get key variables
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SBX_SECRET_KEY = os.getenv('PLAID_SBX_SECRET_KEY')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

### 2. Generate a public token

In [71]:
# Selecting an institution fto process
INSTITUTION_ID = "ins_109512"

# Creating public token key to be swapped for institution access token
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions','income','assets'])
create_tkn_response

{'public_token': 'public-sandbox-c5fb3dce-35d6-4351-a47e-0ea4939900e9',
 'request_id': 'JR9nTlbiJR6emh3'}

### 3. Exchange the public token for an access token

In [72]:
# Exchanging the public token for access token
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])
exchange_response

{'access_token': 'access-sandbox-a3c17c26-6940-4354-ae78-8bfe1891bd50',
 'item_id': 'Rvdj7BJLo5FZXQW8oG4aSGjJ5DX67NTRK15ZX',
 'request_id': 'I3dEWJW7qwLqnAc'}

### 4. Fetch Accounts

In [73]:
# Storing access token as variable
access_token = exchange_response['access_token']
# Get accounts associated with the institution
client.Accounts.get(access_token)

{'accounts': [{'account_id': 'wMaQ43DBJju3qGPxv6Mmcobj8Bqx7gur7xzZl',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': '5PeyW7zZ6gt9GRoJd1MgirdRz7NeJGtZPAXKg',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'JvlB3kdP95FbJDv8yRNBsRjdzgMropudrvX61',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [74]:
# Fetching Transactions for last 90 days
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())
print(json.dumps(transaction_response['transactions'][:2],indent=4, sort_keys=True))

[
    {
        "account_id": "V5kjrQr4oAhvZ1A61qXQHNDwjVg3AMFWQZm3d",
        "account_owner": null,
        "amount": 89.4,
        "authorized_date": null,
        "category": [
            "Food and Drink",
            "Restaurants"
        ],
        "category_id": "13005000",
        "date": "2020-04-11",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "name": "SparkFun",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },
        "pending": false,
     

### 2. Print the categories for each transaction

In [75]:
# Printing categories on each transaction
transaction_response = client.Transactions.get(access_token,start_date,end_date)
for transactions in transaction_response["transactions"]:
    print(json.dumps(transactions["category"][:2],indent=4, sort_keys=True))

[
    "Food and Drink",
    "Restaurants"
]
[
    "Transfer",
    "Credit"
]
[
    "Travel",
    "Airlines and Aviation Services"
]
[
    "Travel",
    "Taxi"
]
[
    "Food and Drink",
    "Restaurants"
]
[
    "Payment"
]
[
    "Food and Drink",
    "Restaurants"
]
[
    "Shops",
    "Sporting Goods"
]
[
    "Payment",
    "Credit Card"
]
[
    "Travel",
    "Taxi"
]
[
    "Transfer",
    "Debit"
]
[
    "Transfer",
    "Deposit"
]
[
    "Recreation",
    "Gyms and Fitness Centers"
]
[
    "Travel",
    "Airlines and Aviation Services"
]
[
    "Food and Drink",
    "Restaurants"
]
[
    "Food and Drink",
    "Restaurants"
]
[
    "Food and Drink",
    "Restaurants"
]
[
    "Transfer",
    "Credit"
]
[
    "Travel",
    "Airlines and Aviation Services"
]
[
    "Travel",
    "Taxi"
]
[
    "Food and Drink",
    "Restaurants"
]
[
    "Payment"
]
[
    "Food and Drink",
    "Restaurants"
]
[
    "Shops",
    "Sporting Goods"
]
[
    "Payment",
    "Credit Card"
]
[
    "Travel",
    "Taxi

### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [76]:
# Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category

transactions=pd.DataFrame()

# Build columns of data frame for transactions
transactions['date'] = pd.Series([transactions['date'] for transactions in transaction_response['transactions']])
transactions['name'] = pd.Series([transactions['name'] for transactions in transaction_response['transactions']])
transactions['amount'] = pd.Series([transactions['amount'] for transactions in transaction_response['transactions']])
transactions['category'] = pd.Series([transactions['category'] for transactions in transaction_response['transactions']])       

#print first five lines of transactions data frame
transactions.head()

,date,name,amount,category
0,2020-04-11,SparkFun,89.40,"[Food and Drink, Restaurants]"
1,2020-04-10,INTRST PYMNT,-4.22,"[Transfer, Credit]"
2,2020-03-31,United Airlines,500.00,"[Travel, Airlines and Aviation Services]"
3,2020-03-29,Uber,6.33,"[Travel, Taxi]"
4,2020-03-26,Tectra Inc,500.00,"[Food and Drink, Restaurants]"


### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [77]:
#seting the index as the date value of transaction
datetimeindex=transactions["date"]
transactions=transactions.set_index(datetimeindex)
transactions.index=pd.to_datetime(transactions.index)
#drop date from dataframe columns
transactions.drop("date",axis=1,inplace=True)
#updated amount column of dataframe 
transactions["amount"]=transactions["amount"].astype(float,inplace=True)
transactions.head()

,name,amount,category
date,,,
2020-04-11,SparkFun,89.40,"[Food and Drink, Restaurants]"
2020-04-10,INTRST PYMNT,-4.22,"[Transfer, Credit]"
2020-03-31,United Airlines,500.00,"[Travel, Airlines and Aviation Services]"
2020-03-29,Uber,6.33,"[Travel, Taxi]"
2020-03-26,Tectra Inc,500.00,"[Food and Drink, Restaurants]"


---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [78]:
# YOUR CODE HERE
# Extract For a year
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-365))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

# Get income
income_response = client.Income.get(access_token)

# Print JSON output
print(json.dumps(income_response,indent=4, sort_keys=True))

{
    "income": {
        "income_streams": [
            {
                "confidence": 0.99,
                "days": 690,
                "monthly_income": 500,
                "name": "UNITED AIRLINES"
            }
        ],
        "last_year_income": 6000,
        "last_year_income_before_tax": 7285,
        "max_number_of_overlapping_income_streams": 1,
        "number_of_income_streams": 1,
        "projected_yearly_income": 6085,
        "projected_yearly_income_before_tax": 7389
    },
    "request_id": "amxHge8sgWQbuLU"
}


---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [79]:
#bring down dataframe for reference
transactions_df.head()

NameError: name 'transactions_df' is not defined

### Calculate the expenses per month

In [60]:
# Expenses per month over past 90 days (3 months 01-Jan, 02-Feb, 03-Mar)
monthly_expense = transactions_df.groupby(transactions_df.index.month).sum()
monthly_expense

NameError: name 'transactions_df' is not defined